<a href="https://colab.research.google.com/github/ParsaRouzrokh/Keras-PlayGround/blob/main/Translator_Transformer_(Eng_Esp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset Download
import tensorflow as tf
keras = tf.keras
import numpy as np
from keras import layers
from random import shuffle
import string
import re

!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-08-31 19:42:05--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  3.82MB/s    in 0.7s    

2023-08-31 19:42:06 (3.82 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [ ]:
# Reading Dataset

with open ("spa-eng/spa.txt") as f:
  lines = f.read().split('\n')[:-1]

# As mentioned in README of the dataset, the last record is empty.

In [ ]:
pairs = []
for line in lines[:42000]:
  eng,esp = line.split('\t')
  # Adding start and end tokens to our target texts
  esp = '[start] '+ esp + ' [end]'

  # Building lists including pairs of eng and esp texts
  pairs.append((eng,esp))
pairs[0]

('Go.', '[start] Ve. [end]')

In [ ]:
# shuffle and split

from random import shuffle
shuffle(pairs)
pairs_length = len (pairs)
val_length = test_length =  int(0.15 * pairs_length)
train_length = pairs_length - 2 * (val_length)
train_pairs = pairs[:train_length]
val_pairs = pairs[train_length:train_length + val_length]
test_pairs = pairs[train_length + val_length:]

In [ ]:
# normalization
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace('[','') # Needed for Tokens!
strip_chars = strip_chars.replace(']','') # Needed for Tokens!
strip_chars # !"#$%&\'()*+,-./:;<=>?@\\^_`{|}~¿

def standard (input_str): # in kar faqat baraye spaniaii bayad anjam beshe yani target !
  lowercase = tf.strings.lower(input_str)

  # Using re.escape to consider special characters as literal ones!
  stn = tf.strings.regex_replace(lowercase , f"[{re.escape(strip_chars)}]" , "")

  return stn

In [ ]:
# Vectorization

# We are using the first 15000 of the most common used words in our corpus.
vocab_size = 15000

# We are setting the lengths of our input and output literal texts (not tokens) to 20!
seq_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length= seq_length)

target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",

    # Since we are using encoder_decoder here and we are using esp texts for \
    # inputs and outputs, the out_seq_length must me 21 here!
    # input_esp:21 => all tokens without [end]
    # target_esp:21 => all tokens without [start]
    output_sequence_length = seq_length + 1,
    standardize = standard
)

# Passing the corpuses of our vocabs to our vectorization layers.

full_train_eng_texts = [pair[0] for pair in train_pairs]
full_train_esp_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(full_train_eng_texts)
target_vectorization.adapt(full_train_esp_texts)


In [ ]:
# Creating our datasets with the correct format for encoder_decoder

batch_size = 64

def format_dataset(eng,spa):
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return({"english" : eng , "spanish" : spa[:,:-1]}, spa[:,1:])

def make_dataset (pairs):
  eng_texts , spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts,spa_texts))
  dataset = dataset.batch (batch_size)
  dataset = dataset.map(format_dataset , num_parallel_calls = 4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset (train_pairs)
val_ds = make_dataset (val_pairs)

In [ ]:
# Sanity Check Here!

for inputs,targets in train_ds.take(1):
  print(f"inputs['english'].shape : {inputs['english'].shape}")
  print(f"inputs['spanish'].shape : {inputs['spanish'].shape}")
  print(f"targets.shape : {targets.shape}")

inputs['english'].shape : (64, 20)
inputs['spanish'].shape : (64, 20)
targets.shape : (64, 20)


In [ ]:
# Lets build our Transformer:
# 1) PositionalEmbedding 2) TransformerEncoder 3) TransformerDecoder


# 1) PositionalEmbedding:
class PositionalEmbedding (layers.Layer):
  def __init__(self, seq_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(input_dim = input_dim , output_dim = output_dim)
    self.position_embeddings = layers.Embedding (input_dim = seq_length , output_dim = output_dim)
    self.seq_length = seq_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start = 0 , limit = length , delta = 1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return (embedded_tokens + embedded_positions)

  # Mask Padding with 0 and 1:
  def compute_mask (self, inputs, mask = None):
    return (tf.math.not_equal(inputs,0))


# Updating the config of our model:
  def get_config(self):
    config = super().get_config()
    config.update({
        "output_dim" : self.output_dim,
        "seq_length" : self.seq_length,
        "input_dim" : self.input_dim
    })
    return config

# Transformer Encoder:

class TransformerEncoder (layers.Layer):
  def __init__(self , embed_dim , dense_dim , num_heads , **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(num_heads = num_heads , key_dim = embed_dim)
    self.dense_proj = keras.Sequential([
        layers.Dense(dense_dim , activation = "relu"),
        layers.Dense(embed_dim)
    ])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()


  def get_config (self):
    config = super().get_config
    config.update({
        "embed_dim": self.embed_dim,
        "dense_dim": self.dense_dim,
        "num_heads": self.num_heads
      })
    return config

  def call(self,inputs, mask = None):
    if mask is not None:
      mask = mask[:,tf.newaxis,:]
    attention_out = self.attention(
        query = inputs,
        key = inputs,
        value = inputs,
        attention_mask = mask
      )
    # Multi_head self_attention for encoder part
    proj_input = self.layernorm_1(attention_out + inputs)
    proj_output = self.dense_proj (proj_input)

    return(self.layernorm_2(proj_input+proj_output))

# Transformer Decoder
class TransformerDecoder (layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)

    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention_1 = layers.MultiHeadAttention(num_heads = num_heads , key_dim = embed_dim)
    self.attention_2 = layers.MultiHeadAttention(num_heads = num_heads , key_dim = embed_dim)
    self.dense_proj = keras.Sequential([
        layers.Dense(dense_dim , activation = "relu"),
        layers.Dense(embed_dim)
    ])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
    self.layernorm_3 = layers.LayerNormalization()

    # Required for passing the custom mask defined in the encoder part, to the decoder part!
    self.supports_masking = True

  def get_config (self):
    config = super().get_config
    config.update({
        "embed_dim": self.embed_dim,
        "dense_dim": self.dense_dim,
        "num_heads": self.num_heads
    })
    return config


  # Since in our transformer architecture, the model is inspecting our whole esp \
  # inputs and targets at the same time, we must create padding masks for these 2!


  def casual_attention_mask(self,inputs):
    input_shape = tf.shape(inputs)
    batch_size , sequence_length = input_shape[0],input_shape[1]
    i = tf.range(sequence_length)[:,tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast (i >= j , dtype = "int32") # Now the lower half of our 20*20 matrix is 1 , rest = 0!
    mask = tf.reshape(mask,(1,sequence_length,sequence_length))
    mult = tf.concat(
        [tf.expand_dims(batch_size,-1),tf.constant([1,1],dtype = tf.int32)], axis = 0)
    return tf.tile(mask,mult)


  def call(self,inputs,encoder_outputs, mask = None):
    casual_mask = self.casual_attention_mask(inputs)
    if mask is not None:
      padding_mask = tf.cast(mask[:,tf.newaxis,:] , dtype ="int32")
      # choosing the lowest value between these two based on our encoder part!
      padding_mask = tf.minimum(padding_mask,casual_mask)
    attention_out1 = self.attention_1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask
        )
    attention_out1 = self.layernorm_1(attention_out1 + inputs)
    attention_out2 = self.attention_2(
            query = attention_out1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask # Prevent Spoilers!
        )
    attention_out2 = self.layernorm_2(attention_out2 + attention_out1)
    proj_out = self.dense_proj(attention_out2)

    return (self.layernorm_3(proj_out + attention_out2))

In [ ]:
# END TO END:

embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape = (None,), dtype = "int64" , name = "english")
x = PositionalEmbedding(seq_length, vocab_size, embed_dim) (encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim , num_heads) (x)

decoder_inputs = keras.Input(shape = (None,) , dtype ="int64", name = "spanish")
x = PositionalEmbedding(seq_length, vocab_size, embed_dim) (decoder_inputs)
x = TransformerDecoder (embed_dim, dense_dim , num_heads)(x,encoder_outputs)
decoder_outputs = layers.Dense(vocab_size , activation = "softmax")(x)
Transformer = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)
Transformer.compile(optimizer = "rmsprop" , loss = "sparse_categorical_crossentropy" , metrics = ["accuracy"])

history = Transformer.fit(train_ds , epochs = 30 , validation_data = val_ds)

Epoch 1/30
460/460 [==============================] - 53s 87ms/step - loss: 3.7043 - accuracy: 0.4833 - val_loss: 2.9105 - val_accuracy: 0.5586
Epoch 2/30
460/460 [==============================] - 30s 65ms/step - loss: 2.4758 - accuracy: 0.6046 - val_loss: 2.4645 - val_accuracy: 0.6105
Epoch 3/30
460/460 [==============================] - 30s 66ms/step - loss: 1.9335 - accuracy: 0.6698 - val_loss: 2.2034 - val_accuracy: 0.6457
Epoch 4/30
460/460 [==============================] - 31s 67ms/step - loss: 1.5680 - accuracy: 0.7168 - val_loss: 2.0620 - val_accuracy: 0.6661
Epoch 5/30
460/460 [==============================] - 31s 67ms/step - loss: 1.3014 - accuracy: 0.7521 - val_loss: 2.0604 - val_accuracy: 0.6712
Epoch 6/30
460/460 [==============================] - 31s 67ms/step - loss: 1.0934 - accuracy: 0.7818 - val_loss: 2.0278 - val_accuracy: 0.6754
Epoch 7/30
460/460 [==============================] - 31s 67ms/step - loss: 0.9244 - accuracy: 0.8067 - val_loss: 2.1306 - val_accuracy:

In [ ]:
# It's always better not to trust any metric in translations except yourself!

spa_vocab = target_vectorization.get_vocabulary()

# Creating Dictionary of Spanish word indices and the words themselves!
spa_index_lookup = dict(zip(range(len(spa_vocab)),spa_vocab))

max_decoded_length = 20

def decode_sequence (input_sentence):
  tokenized_input = source_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range (max_decoded_length):
    tokenized_output = target_vectorization([decoded_sentence])[:,:-1]
    next_token_pred = Transformer.predict([tokenized_input,tokenized_output])
    sampled_token_index = np.argmax(next_token_pred[:,i,:])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "end":
      break
  return str(decoded_sentence).split("[start]")[1].split("[end]")[0].strip()



In [ ]:
"""for _ in range(20):
  input_sen = random.choice(full_train_eng_texts)
  print("-")
  print(input_sen)
  print(decode_sequence(input_sen))"""
print(decode_sequence("what's your name"))

1/1 [==============================] - 0s 31ms/step
cómo te llamas
